In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [2]:
x = np.loadtxt('./data/2Dpoints.txt')

In [4]:
X = np.loadtxt('./data/3Dpoints.txt')

In [6]:
len(X)

100

In [21]:
X.shape

(100, 3)

In [14]:
a = np.hstack((X[0], [1]))
b = np.zeros(a.shape)

In [17]:
print(np.vstack((a,b)))

[[ 8.89560004e+03 -3.42725524e+02  7.29791079e+03  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [25]:
t1 = np.zeros((2*len(X), 4))
t2 = np.zeros((2*len(X), 4))
for i in range(len(X)):
    t1[i*2] = np.hstack((X[i], [1]))
    t2[i*2+1] = np.hstack((X[i], [1]))

In [29]:
b1 = np.zeros((2*len(X), 4))
b2 = np.zeros((2*len(X), 4))
for i in range(len(x)):
    xi, yi = x[i]
    b1[i*2] = np.hstack((-xi*X[i], [-xi]))
    b2[i*2+1] = np.hstack((-yi*X[i], [-yi]))

In [30]:
b = b1+b2

In [33]:
A = np.hstack((t1, t2, b))

In [34]:
A.shape

(200, 12)

In [45]:
ew, ev = np.linalg.eig(A.T @ A)

In [111]:
p = ev[-1]

In [112]:
p /= np.linalg.norm(p)

In [113]:
np.linalg.norm(p)

1.0

In [114]:
p = p.reshape((3,4))

In [115]:
p

array([[ 1.05618666e-04, -1.17993320e-05,  2.54052806e-05,
        -9.70520465e-05],
       [ 6.21564806e-04, -4.63279770e-04,  7.33643729e-03,
        -8.54249654e-03],
       [-9.45349262e-01,  3.25860515e-01, -6.69678660e-04,
        -1.35235046e-03]])

## Load the points

`The function gets rid of the annoying error that NumPy's loadtxt() cries about.`

In [63]:
def fileprocess(s):
    im1, im2 = [], []
    with open(s) as file:
        for line in file:
            a, b, c, d = line.split(',')
            im1.append([float(a),float(b)])
            im2.append([float(c),float(d)])
    return im1,im2

In [82]:
img1, img2 = fileprocess('./data/homography.txt')

In [83]:
img1, img2 = np.array(img1), np.array(img2)

### Compute $T^a$ and $T^b$

In [89]:
def computeTransform(img):
    x = img[:,0]
    y = img[:,1]

    T = np.zeros((3,3))
    
    s = np.sqrt(2)/ np.mean(np.sqrt((x-x.mean())**2 + (y-y.mean())**2))
    
    T[0] = np.array([s, 0, -s*x.mean()])
    T[1] = np.array([0, s, -s*y.mean()])
    T[2] = np.array([0, 0, 1])
    
    return T, s

In [90]:
Ta, s1 = computeTransform(img1)

In [91]:
Tb, s2 = computeTransform(img2)

### Normalize Data

In [93]:
def normalize(img, s):
    x = img[:,0]
    y = img[:,1]
    x -= x.mean()
    y -= y.mean()    
    img[:,0] = s*x
    img[:,1] = s*y
    
    return img

In [94]:
img1_n = normalize(img1, s1)
img2_n = normalize(img2, s2)

### Compute $\tilde{H}$

In [98]:
x1 = img1_n[:,0]
y1 = img1_n[:,1]

x2 = img2_n[:,0]
y2 = img2_n[:,1]

In [99]:
ht1 = np.zeros((2*len(x1), 3))
ht2 = np.zeros((2*len(x1), 3))
ht3 = np.zeros((2*len(x1), 3))
for i in range(len(x1)):
    ht1[i*2] = np.hstack((x1[i], y1[i], 1))
    ht2[i*2+1] = np.hstack((x1[i], y1[i], 1))
    ht3[i*2] = np.hstack((-x2[i]*x1[i], -y1[i]*x2[i], -x2[i]))
    ht3[i*2+1] = np.hstack((-y2[i]*x1[i], -y1[i]*y2[i], -y2[i]))

In [101]:
AH_bar = np.hstack((ht1, ht2, ht3))

In [102]:
ew2, ev2 = np.linalg.eig(AH_bar.T @ AH_bar)

In [103]:
H_bar = ev2[-1]

In [104]:
H_bar /= np.linalg.norm(H_bar)

In [105]:
np.linalg.norm(H_bar)

0.9999999999999999

In [106]:
H_bar = H_bar.reshape((3,3))

In [107]:
H_bar

array([[-0.09512031, -0.77332211, -0.02229385],
       [ 0.01220048,  0.55951519,  0.16545529],
       [-0.09442806, -0.20352084, -0.0388468 ]])

### Compute $H$

In [117]:
H = np.linalg.inv(Tb) @ H_bar @ Ta

In [118]:
H

array([[-0.09512031, -0.77332211, -0.02229385],
       [ 0.01220048,  0.55951519,  0.16545529],
       [-0.09442806, -0.20352084, -0.0388468 ]])